<center><img src="https://github.com/FASSt-simulation/fasst_simulation_tools/raw/main/images/site-logo.png" width=150 height=150 alt="NGEE-Arctic Logo"/></center>

# An example of reading in and plotting one variable from ELM output.

#### ----------------------------------------------------------------------------------------------------------------------

<br>

To run this script and generate the resulting ELM plots we need specific Python libraries.  This step loads those required libraries

In [ ]:
# Load the required Python libraries

import matplotlib.pyplot as plt
import xarray
from netCDF4 import Dataset 
import os
import cftime
import ipywidgets as widgets
import glob, numpy

In [ ]:
# Create case run output directories

output_rootdir=os.path.expanduser('~')+'/output/cime_run_dirs/'
cases=numpy.asarray(glob.glob("%s*20TR*" % output_rootdir))
cases=[x.split('/')[-1] for x in cases]
cases_dropdown = widgets.Dropdown(options=cases,
                                description='Choose Case Name:',
                                style={'description_width':'auto'},
                                layout={'width':'max-content'},
                                disabled=False)

In [ ]:
# Show a dropdown menu to select specific case output
# Any cases that have been run in our elmoutput directory will be displayed
# Case names will contain the site codes:
# AK-BEO; AK-CLG; AK-K64G AK-TLG

display(cases_dropdown)

In [ ]:
# Get the output nc file options

output_casedir=output_rootdir+cases_dropdown.value+'/run/'
filenames=numpy.asarray(glob.glob("%s*.elm.h?.*.nc" % output_casedir))
filenames=sorted([x.split('/')[-1] for x in filenames])
if(os.path.exists(output_casedir+'ELM_output.nc')): filenames.insert(0,'ELM_output.nc')
if(os.path.exists(output_casedir+'ELM_output_PFT.nc')): filenames.insert(0,'ELM_output_PFT.nc')

ncfiles_dropdown = widgets.Dropdown(options=filenames,
                                description='Choose Output File:', 
                                style={'description_width':'auto'},
                                layout={'width':'max-content'},
                                disabled=False)

In [ ]:
# Display the output nc file options - in most cases you will select ELM_output.nc

display(ncfiles_dropdown)

In [ ]:
# Set the output file

output_file=output_casedir+ncfiles_dropdown.value

In [ ]:
# Create list of all output variables contained in the selected output_file

fnc=Dataset(output_file)
vars=fnc.variables
vars_names=vars.keys()
vars_dims={}; vars_units={};
for v in vars: 
    vars_dims[v]=vars[v].dimensions
    if(hasattr(vars[v], 'units')):
        vars_units[v]=vars[v].units
    else:
        vars_units[v]='-'    
fnc.close()
vars_dropdown = widgets.Dropdown(options=vars_names,
                                description='Choose Variable Name',
                                style={'description_width':'auto'},
                                value='GPP',
                                layout={'width':'max-content'},
                                disabled=False)

In [ ]:
# Display the output variables contained in output_file
# Select the variable of interest

display(vars_dropdown)

In [ ]:
# Setup plotting options for selected variable

### user inputs for variable NAME, UNIT, scaling, Year starting and ending for plotting

# user-forced, i.e. NOT by drop-down above,varialbe name in ELM_output.nc to plot
variable_name_user = ''  # leave blank to plot variable selected in dropdown menu
#variable_name_user = 'TLAI'  # or comment out above and insert specific variable name here

# error handling
if((vars_dropdown.value=='' or 'time' not in vars_dims[vars_dropdown.value]) \
   and variable_name_user==''):
    variable_name='GPP'          
elif(variable_name_user==''):
    variable_name=vars_dropdown.value
else:
    variable_name=variable_name_user


variable_unit=''             # user-defined units, '' for that from ELM_output.nc
#variable_unit='ugC/m2/s'     # user-defined units, '' for that from ELM_output.nc

variable_multiplier=1         # scaling original data by multiplying this scalor
#variable_multiplier=1.e6     # scaling original data by multiplying this scalor

yr_start = -9999              # -9999 for default start year in ELM_output.nc
#yr_start = 1850              # -9999 for default start year in ELM_output.nc
yr_end = -9999               # -9999 for default end year in ELM_output.nc
#yr_end = 1850               # -9999 for default end year in ELM_output.nc

In [ ]:
# Open the selected ELM output file

output=xarray.open_dataset(output_file).squeeze()

In [ ]:
# Set the plotting options selected above

if 'time' in vars_dims[variable_name]:
    time = output['time']  ### time dimension, put in unit of years
    if(yr_start!=-9999):
        t0 = cftime.DatetimeNoLeap(yr_start, 1, 1, 0, 0, 0, 0, has_year_zero=True)
        time=time.where(time.time>t0, drop=True)
    if(yr_end!=-9999):
        t1 = cftime.DatetimeNoLeap(yr_end+1, 1, 1, 0, 0, 0, 0, has_year_zero=True)
        time=time.where(time.time<t1, drop=True)
else:
    print("NOT a time-series variable: ", variable_name)

grc_num=1
if 'lndgrid' in output:
    grc_num=output['lndgrid']
    #print(grc_num)
if 'levgrnd' in vars_dims[variable_name]:
    soil_layer=output['levgrnd']
    #print(soil_layer)
if 'levdcmp' in vars_dims[variable_name]:
    soil2_layer=output['levdcmp']
    #print(soil2_layer)
if 'pft' in vars_dims[variable_name]:
    pft_index=output['pft']
    #print(pft_index)

var_data = output[variable_name]*variable_multiplier

if(yr_start!=-9999 and 'time' in vars_dims[variable_name]):
    t0 = cftime.DatetimeNoLeap(yr_start, 1, 1, 0, 0, 0, 0, has_year_zero=True)
    var_data=var_data.where(var_data.time>t0, drop=True)
if(yr_end!=-9999 and 'time' in vars_dims[variable_name]):
    t1 = cftime.DatetimeNoLeap(yr_end+1, 1, 1, 0, 0, 0, 0, has_year_zero=True)
    var_data=var_data.where(var_data.time<t1, drop=True)



if variable_unit=='':
    var_unit = vars_units[variable_name]
    if variable_multiplier!=1: var_unit=str(1.0/variable_multiplier)+var_unit
else:
    var_unit = variable_unit


In [ ]:
# Create the variable plot

if ('lndgrid' in vars_dims[variable_name] and grc_num>1):
    grc_index = 0  # user-editable
    plt.plot(time,var_data[...,grc_index], label=variable_name)
elif ('column' in vars_dims[variable_name]):
    col_index = 0  # user-editable
    plt.plot(time,var_data[...,col_index], label=variable_name)
else:
    plt.plot(time, var_data, label=variable_name)

#if multiple PFTs, pick its index, e.g. 11 (started from 0)
#plt.plot(time, var_data[...,11], label=variable_name)

#if layered soil variables, pick specified layer, e.g. 4 (started from 0)
#plt.plot(time, var_data[...,4], label=variable_name)

plt.xlabel(''.join(['Time (YYYY-MM-DD)']))
plt.ylabel(''.join([variable_name,' (',var_unit, ')']))

<center><img src="https://github.com/FASSt-simulation/fasst_simulation_tools/raw/main/images/site-logo.png" width=150 height=150 alt="NGEE-Arctic Logo"/></center>